In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# Designate variables
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o"
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages= True,
)

loader = TextLoader(r"./files/chapter_3.txt")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./cache/")

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir    
)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

# List of docs generation
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim
    ------------------
    {portion}
    """),
    ("human", "{question}"),
])
map_doc_chain = map_doc_prompt | llm

# Merge list of docs
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({
        "portion": doc.page_content,
        "question": question
    }).content for doc in documents)

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

# Final document | prompt | llm
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Give the following extracted parts of a long document and a question, create a final answer.
    If you don't know the answer, just say that you don't know. Don't try to make up the answer.
    ---------
    {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = {"context": map_chain, "question": RunnablePassthrough()} | RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result=chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")


C:\Users\dlaru\AppData\Local\Temp\ipykernel_3596\4105870871.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
C:\Users\dlaru\AppData\Local\Temp\ipykernel_3596\4105870871.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
C:\Users\dlaru\AppData\Local\Temp\ipykernel_3596\4105870871.py:31: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~lan

content='Based on the information provided, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 137, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-d836616f-4dd5-4d7b-8cec-aaa42c7eb5c7-0'


In [2]:
invoke_chain("What message did he write in the table?")

content='The message he wrote on the table was:\n\nFREEDOM IS SLAVERY\n\nTWO AND TWO MAKE FIVE\n\nGOD IS POWER' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 186, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-676f751b-daf6-4e65-ae81-b9a88678ddc1-0'


In [3]:
invoke_chain("Who is Julia?")

content="I'm sorry, but the provided text does not contain information about who Julia is." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 195, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_9faba9f038', 'finish_reason': 'stop', 'logprobs': None} id='run-f06d4ea0-76e0-49ad-848c-66460f657137-0'
